In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# Data Analysis with Python III - Validating and Cleaning Data

<!-- PELICAN_BEGIN_SUMMARY -->
A favorite old saying of mine with respect to data quality is "Garbage In and Garbage Out".
<br>My objective here is to go through some examples for how to perform basic data validation.
<!-- PELICAN_END_SUMMARY -->

**Goals: How to get a clean and valid dataset?**
- Join two or more dataframes by using a key or composite key
- Spot invalid data by using pandas 
   - Pandas offers functionality similar to Excel Vlookup, Sumif etc.
- Remove duplicate data
- Calculate the average value or the maximum value of certain dataset

I plan to demonstrate how to validate data using a "dirty" dataset.
<br>Specifically, how to 
<br>- detect/eliminate outlier data
<br>- detect/eliminate duplicate data
<br>- deal with missing data
<br>- add default values
<br>- remove incomplete data/rows
<br>- deal with error-prone columns
<br>- normalize data types
<br>- change casing
<br>- rename columns
<br>- make bulk corrections to string/number values where appropriate

In [ ]:
**Excel Index-Match function in python**

In [3]:
x = pd.DataFrame(data=list(range(0, 10)), columns=["List"])
x
y = pd.DataFrame(data=[(-1, "Small"), (4, "Medium"), (7, "Large")], 
                         columns=["List", "Category"])
y
pd.cut(x.List, y.List.tolist()+[np.inf], labels=y.Category)

,List
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


,List,Category
0,-1,Small
1,4,Medium
2,7,Large


0     Small
1     Small
2     Small
3     Small
4     Small
5    Medium
6    Medium
7    Medium
8     Large
9     Large
Name: List, dtype: category
Categories (3, object): [Small < Medium < Large]